In [1]:
import warnings
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.util
import iris
import iris.analysis
import iris.coord_categorisation
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter

In [2]:
from commons import EXPERIMENTS, LAYERS
from mypaths import path_to_ukca  # path_to_figs
from plot_func import use_draft_style
from proc_func import GASES

In [3]:
warnings.filterwarnings("ignore", module="iris")
use_draft_style()

In [4]:
# Choose experiments
base_exp = "BASE"
sens_exp = "MARI"
# Choose layer
layer = "boundary"  # boundary or free troposphere

In [5]:
base_cl = iris.cube.CubeList()
sens_cl = iris.cube.CubeList()
for igas, gasdict in GASES.items():
    if gasdict["noy"]:
        # Load NOy species data and convert it to ppb
        base_cb = (
            iris.load_cube(
                str(path_to_ukca / EXPERIMENTS[base_exp] / f"{EXPERIMENTS[base_exp]}_{igas}.nc"),
                f"{igas}",
            )
            * GASES[igas]["mmr_to_vmr"]
            * 1e9
        )
        sens_cb = (
            iris.load_cube(
                str(path_to_ukca / EXPERIMENTS[sens_exp] / f"{EXPERIMENTS[sens_exp]}_{igas}.nc"),
                f"{igas}",
            )
            * GASES[igas]["mmr_to_vmr"]
            * 1e9
        )
        base_cb.rename(igas)
        sens_cb.rename(igas)

        # Remove the first 2 years of data as a spin up
        base_cb_trimmed = base_cb[24::, ...]
        sens_cb_trimmed = sens_cb[24::, ...]

        base_cl.append(base_cb_trimmed)
        sens_cl.append(sens_cb_trimmed)

# Horizontal grid
lons = base_cb.coord("longitude").points
lats = base_cb.coord("latitude").points

In [6]:
# Calculate NOy
base_noy = sum(base_cl)
sens_noy = sum(sens_cl)
base_noy.rename("noy")
sens_noy.rename("noy")

# Add additional time coordinates
iris.coord_categorisation.add_season(base_noy, "time", name="season")
iris.coord_categorisation.add_season(sens_noy, "time", name="season")
iris.coord_categorisation.add_season_year(base_noy, "time", name="year")
iris.coord_categorisation.add_season_year(sens_noy, "time", name="year")

In [7]:
# Calculate RONO2 sum
base_rono2 = sum(base_cl.extract(["meono2", "etono2", "nprono2", "iprono2"]))
sens_rono2 = sum(sens_cl.extract(["meono2", "etono2", "nprono2", "iprono2"]))
base_rono2.rename("rono2")
sens_rono2.rename("rono2")

# Note: RONO2 concentrations are zero in the BASE experiment

In [8]:
seasons = ["djf", "mam", "jja", "son"]

# Calculate seasonal means in a specified layer

# def calc_layer_seasonal_mean():


base_noy_by_season = iris.cube.CubeList()
sens_noy_by_season = iris.cube.CubeList()
for iseason in seasons:
    icb = base_noy.extract(iris.Constraint(season=iseason) & LAYERS[layer]).collapsed(
        ["season", "level_height"], iris.analysis.MEAN
    )
    jcb = sens_noy.extract(iris.Constraint(season=iseason) & LAYERS[layer]).collapsed(
        ["season", "level_height"], iris.analysis.MEAN
    )
    base_noy_by_season.append(icb)
    sens_noy_by_season.append(jcb)

In [9]:
break

SyntaxError: 'break' outside loop (<ipython-input-9-6aaf1f276005>, line 4)

In [ ]:
import aeolus

In [ ]:
# Add cyclic point for plotting on a global map
cyc_rno3_bl_djf_mean, cyclic_lons = cartopy.util.add_cyclic_point(
    rno3_bl_djf_mean.data, coord=rno3_bl_djf_mean.coord("longitude").points
)
cyc_rno3_bl_mam_mean = cartopy.util.add_cyclic_point(rno3_bl_mam_mean.data)
cyc_rno3_bl_jja_mean = cartopy.util.add_cyclic_point(rno3_bl_jja_mean.data)
cyc_rno3_bl_son_mean = cartopy.util.add_cyclic_point(rno3_bl_son_mean.data)
# Find max boundary layer seasonal mean
print(np.max(rno3_bl_djf_mean.data) * 1e3)
print(np.max(rno3_bl_mam_mean.data) * 1e3)
print(np.max(rno3_bl_jja_mean.data) * 1e3)
print(np.max(rno3_bl_son_mean.data) * 1e3)
# rno3_mean_cf_kwargs = dict(transform=ccrs.PlateCarree(), levels=np.arange(0,110,10))

In [ ]:
fig, ax = plt.subplots(
    nrows=4,
    ncols=2,
    figsize=(12, 12),
    subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0)),
    facecolor="w",
)
p00 = ax[0, 0].contourf(cyclic_lons, lats, cyc_sens_bl_djf_mean.data, transform=ccrs.PlateCarree())
ax[1, 0].contourf(cyclic_lons, lats, cyc_sens_bl_mam_mean.data, transform=ccrs.PlateCarree())
ax[2, 0].contourf(cyclic_lons, lats, cyc_sens_bl_jja_mean.data, transform=ccrs.PlateCarree())
ax[3, 0].contourf(cyclic_lons, lats, cyc_sens_bl_son_mean.data, transform=ccrs.PlateCarree())

p01 = ax[0, 1].contourf(
    cyclic_lons,
    lats,
    cyc_rno3_bl_djf_mean.data * 100 / cyc_sens_bl_djf_mean.data,
    transform=ccrs.PlateCarree(),
    levels=np.arange(0, 110, 10),
)
ax[1, 1].contourf(
    cyclic_lons,
    lats,
    cyc_rno3_bl_mam_mean.data * 100 / cyc_sens_bl_mam_mean.data,
    transform=ccrs.PlateCarree(),
    levels=np.arange(0, 110, 10),
)
ax[2, 1].contourf(
    cyclic_lons,
    lats,
    cyc_rno3_bl_jja_mean.data * 100 / cyc_sens_bl_jja_mean.data,
    transform=ccrs.PlateCarree(),
    levels=np.arange(0, 110, 10),
)
ax[3, 1].contourf(
    cyclic_lons,
    lats,
    cyc_rno3_bl_son_mean.data * 100 / cyc_sens_bl_son_mean.data,
    transform=ccrs.PlateCarree(),
    levels=np.arange(0, 110, 10),
)

p01x = ax[0, 1].contour(
    lons,
    lats,
    rno3_bl_djf_mean.data * 100 / sens_bl_djf_mean.data,
    transform=ccrs.PlateCarree(),
    levels=[10, 50, 80],
    colors="w",
)
p11x = ax[1, 1].contour(
    lons,
    lats,
    rno3_bl_mam_mean.data * 100 / sens_bl_mam_mean.data,
    transform=ccrs.PlateCarree(),
    levels=[10, 50, 80],
    colors="w",
)
p21x = ax[2, 1].contour(
    lons,
    lats,
    rno3_bl_jja_mean.data * 100 / sens_bl_jja_mean.data,
    transform=ccrs.PlateCarree(),
    levels=[10, 50, 80],
    colors="w",
)
p31x = ax[3, 1].contour(
    lons,
    lats,
    rno3_bl_son_mean.data * 100 / sens_bl_son_mean.data,
    transform=ccrs.PlateCarree(),
    levels=[10, 50, 80],
    colors="w",
)
ax[0, 1].clabel(p01x, inline=1, fmt="%1.0f")
ax[1, 1].clabel(p11x, inline=1, fmt="%1.0f")
ax[2, 1].clabel(p21x, inline=1, fmt="%1.0f")
ax[3, 1].clabel(p31x, inline=1, fmt="%1.0f")

fig.subplots_adjust(hspace=0.1, wspace=-0.1)
cax00 = fig.add_axes([0.215, 0.1, 0.23, 0.01])
fig.colorbar(p00, cax=cax00, orientation="horizontal", label="$NO_y$, ppbv")
cax01 = fig.add_axes([0.58, 0.1, 0.23, 0.01])
fig.colorbar(
    p01, cax=cax01, orientation="horizontal", label="$\sum$$C_1$-$C_3$ $RONO_2$ % of $NO_y$"
)
fig.suptitle(f"Boundary layer (0-2 km)\n{sens_exp}", y=0.92, weight="bold")
for iax in ax.flatten():
    iax.coastlines(color="k")